In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Análisis Visual del Negocio - Airbnb NYC\n",
    "\n",
    "Este notebook se conecta a las tablas pre-procesadas en la capa `business_layer` de nuestro Data Warehouse en Google BigQuery para visualizar las respuestas a las preguntas de negocio clave."
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. Configuración e Importación de Librerías"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import os\n",
    "import pandas as pd\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "from google.cloud import bigquery\n",
    "\n",
    "# --- Configuración del Proyecto ---\n",
    "GCP_PROJECT_ID = \"eng-name-468100-g3\"\n",
    "CREDENTIALS_PATH = 'gcp_credentials.json'\n",
    "\n",
    "# --- Configuración de Estilo para los Gráficos ---\n",
    "sns.set_theme(style=\"whitegrid\")\n",
    "plt.style.use('seaborn-v0_8-talk')\n",
    "\n",
    "# --- Autenticación con Google Cloud ---\n",
    "os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = CREDENTIALS_PATH\n",
    "client = bigquery.Client(project=GCP_PROJECT_ID)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. ¿Cuál es el precio promedio por barrio y distrito?"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "query_avg_price = f\"\"\"\n",
    "    SELECT * \n",
    "    FROM `{GCP_PROJECT_ID}.business_layer.avg_price_by_neighbourhood`\n",
    "    ORDER BY avg_price_usd DESC\n",
    "    LIMIT 15\n",
    "\"\"\"\n",
    "\n",
    "df_avg_price = client.query(query_avg_price).to_dataframe()\n",
    "\n",
    "plt.figure(figsize=(12, 8))\n",
    "sns.barplot(x='avg_price_usd', y='neighbourhood', data=df_avg_price, palette='viridis', hue='neighbourhood_group', dodge=False)\n",
    "plt.title('Top 15 Barrios con Mayor Precio Promedio (USD)')\n",
    "plt.xlabel('Precio Promedio (USD)')\n",
    "plt.ylabel('Barrio')\n",
    "plt.tight_layout()\n",
    "plt.show()\n",
    "\n",
    "print(\"Datos de los 5 barrios más caros:\")\n",
    "display(df_avg_price.head())"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "**Análisis:** Claramente, Manhattan domina los precios más altos, con barrios como Tribeca y el Distrito Financiero a la cabeza. Esto confirma la percepción de que es el distrito más caro para alojarse."
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. ¿Qué tipo de habitación es el más ofrecido y cuál genera mayor revenue?"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "query_room_type = f\"\"\"\n",
    "    SELECT * \n",
    "    FROM `{GCP_PROJECT_ID}.business_layer.room_type_analysis`\n",
    "\"\"\"\n",
    "df_room_type = client.query(query_room_type).to_dataframe()\n",
    "\n",
    "fig, axes = plt.subplots(1, 2, figsize=(18, 7))\n",
    "\n",
    "# Gráfico de Total de Listados\n",
    "sns.barplot(ax=axes[0], x='room_type', y='total_listings', data=df_room_type, palette='magma')\n",
    "axes[0].set_title('Número de Alojamientos por Tipo de Habitación')\n",
    "axes[0].set_xlabel('Tipo de Habitación')\n",
    "axes[0].set_ylabel('Cantidad de Alojamientos')\n",
    "axes[0].tick_params(axis='x', rotation=45)\n",
    "\n",
    "# Gráfico de Revenue Estimado\n",
    "sns.barplot(ax=axes[1], x='room_type', y='estimated_revenue_usd', data=df_room_type, palette='plasma')\n",
    "axes[1].set_title('Revenue Estimado (USD) por Tipo de Habitación')\n",
    "axes[1].set_xlabel('Tipo de Habitación')\n",
    "axes[1].set_ylabel('Revenue Estimado (USD)')\n",
    "axes[1].tick_params(axis='x', rotation=45)\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()\n",
    "\n",
    "print(\"Datos del análisis por tipo de habitación:\")\n",
    "display(df_room_type)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "**Análisis:** Aunque los 'Apartamentos/Casas Enteras' y las 'Habitaciones Privadas' tienen una cantidad similar de listados, los apartamentos enteros generan un revenue estimado significativamente mayor. Esto indica que son más caros o tienen una mayor tasa de ocupación."
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. ¿Cuáles son los anfitriones con más propiedades?"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "query_top_hosts = f\"\"\"\n",
    "    SELECT host_id, host_name, total_listings, avg_price_usd\n",
    "    FROM `{GCP_PROJECT_ID}.business_layer.top_hosts_analysis`\n",
    "    ORDER BY total_listings DESC\n",
    "\"\"\"\n",
    "df_top_hosts = client.query(query_top_hosts).to_dataframe()\n",
    "\n",
    "print(\"Top 20 Anfitriones con más Propiedades Listadas:\")\n",
    "display(df_top_hosts)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "**Análisis:** La tabla revela que el mercado no está atomizado, sino que existen 'power users' o empresas (como Sonder, Blueground) que gestionan cientos de propiedades, operando a una escala muy diferente a la de un anfitrión individual."
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5. ¿Cómo ha sido la evolución de reseñas por mes?"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "query_reviews = f\"\"\"\n",
    "    SELECT *\n",
    "    FROM `{GCP_PROJECT_ID}.business_layer.reviews_evolution_by_month`\n",
    "    WHERE review_year > 2017 -- Filtramos para ver tendencias más recientes\n",
    "\"\"\"\n",
    "df_reviews = client.query(query_reviews).to_dataframe()\n",
    "\n",
    "# Creamos una columna de fecha para graficar correctamente\n",
    "df_reviews['review_date'] = pd.to_datetime(df_reviews['review_year'].astype(str) + '-' + df_reviews['review_month'].astype(str) + '-01')\n",
    "\n",
    "plt.figure(figsize=(15, 8))\n",
    "sns.lineplot(x='review_date', y='total_reviews', hue='neighbourhood_group', data=df_reviews, lw=3)\n",
    "plt.title('Evolución Mensual de Reseñas por Distrito (2018 en adelante)')\n",
    "plt.xlabel('Fecha')\n",
    "plt.ylabel('Cantidad de Reseñas')\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "**Análisis:** El gráfico muestra un crecimiento constante en el número de reseñas hasta mediados de 2019, lo que sugiere un mercado en auge. Brooklyn y Manhattan son claramente los líderes en actividad. Se observa la estacionalidad típica, con picos en los meses de verano."
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.10.0"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}
